In [5]:
import argparse

def parse_args():
    parser = argparse.ArgumentParser("Example of XuanCe.")
    parser.add_argument("--method", type=str, default="ppo")
    parser.add_argument("--env", type=str, default="Box2D")
    parser.add_argument("--env-id", type=str, default="LunarLander-v2")
    # parser.add_argument("--env-id", type=str, default="Pendulum-v1")
    parser.add_argument("--test", type=int, default=0)
    parser.add_argument("--device", type=str, default="cuda")
    parser.add_argument("--benchmark", type=int, default=1)
    parser.add_argument("--config", type=str, default="./configs/lular.yaml")

    return parser.parse_known_args()[0]

In [6]:

import os
from copy import deepcopy
import numpy as np
import torch.optim

from xuance.common import space2shape
from xuance.environment import make_envs
from xuance.torch.utils.operations import set_seed
from xuance.torch.utils import ActivationFunctions
from xuance import get_arguments

In [7]:
parser = parse_args()
args = get_arguments(method=parser.method,
                 env=parser.env,
                 env_id=parser.env_id,
                 config_path=parser.config,
                 parser_args=parser)


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\project\\github\\xuance\\xuance\\configs\\best_Lunar_C51.yaml'

In [ ]:
import os
from copy import deepcopy
import numpy as np
import torch.optim

from xuance.common import space2shape
from xuance.environment import make_envs
from xuance.torch.utils.operations import set_seed
from xuance.torch.utils import ActivationFunctions

def run(args):
    agent_name = args.agent  # get the name of Agent.
    set_seed(args.seed)  # set random seed.

    # prepare directories for results
    args.model_dir = os.path.join(os.getcwd(), args.model_dir, args.env_id)  # the path for saved model.
    args.log_dir = os.path.join(args.log_dir, args.env_id)  # the path for logger file.

    # build environments
    envs = make_envs(args)  # create simulation environments
    args.observation_space = envs.observation_space  # get observation space
    args.action_space = envs.action_space  # get action space
    n_envs = envs.num_envs  # get the number of vectorized environments.
    
    # prepare representation
    from xuance.torch.representations import Basic_MLP
    representation = Basic_MLP(input_shape=space2shape(args.observation_space),
                               hidden_sizes=args.representation_hidden_size,
                               normalize=None,
                               initialize=torch.nn.init.orthogonal_,
                               activation=ActivationFunctions[args.activation],
                               device=args.device)  # create representation

    # prepare policy
    from xuance.torch.policies import Gaussian_AC_Policy
    policy = Gaussian_AC_Policy(action_space=args.action_space,
                                representation=representation,
                                actor_hidden_size=args.actor_hidden_size,
                                critic_hidden_size=args.critic_hidden_size,
                                normalize=None,
                                initialize=torch.nn.init.orthogonal_,
                                activation=ActivationFunctions[args.activation],
                                device=args.device)  # create Gaussian policy

    # prepare agent
    from xuance.torch.agents import PPOCLIP_Agent, get_total_iters
    optimizer = torch.optim.Adam(policy.parameters(), args.learning_rate, eps=1e-5)  # create optimizer
    lr_scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.0,
                                                    total_iters=get_total_iters(agent_name, args))  # for learning rate decay
    agent = PPOCLIP_Agent(config=args,
                          envs=envs,
                          policy=policy,
                          optimizer=optimizer,
                          scheduler=lr_scheduler,
                          device=args.device)  # create a PPO agent

    # start running
    envs.reset()  # reset the environments
    if args.benchmark:  # run benchmark
        def env_fn():  # for creating testing environments
            args_test = deepcopy(args)
            args_test.parallels = args_test.test_episode  # set number of testing environments.
            return make_envs(args_test)  # make testing environments.

        train_steps = args.running_steps // n_envs  # calculate the total running steps.
        eval_interval = args.eval_interval // n_envs  # calculate the number of training steps per epoch.
        test_episode = args.test_episode  # calculate the number of testing episodes.
        num_epoch = int(train_steps / eval_interval)  # calculate the number of epochs.

        test_scores = agent.test(env_fn, test_episode)  # first test
        best_scores_info = {"mean": np.mean(test_scores),  # average episode scores.
                            "std": np.std(test_scores),  # the standard deviation of the episode scores.
                            "step": agent.current_step}  # current step
        for i_epoch in range(num_epoch):  # begin benchmarking
            print("Epoch: %d/%d:" % (i_epoch, num_epoch))
            agent.train(eval_interval)  # train the model for some steps
            test_scores = agent.test(env_fn, test_episode)  # test the model for some episodes

            if np.mean(test_scores) > best_scores_info["mean"]:  # if current score is better than history
                best_scores_info = {"mean": np.mean(test_scores),
                                    "std": np.std(test_scores),
                                    "step": agent.current_step}
                # save best model
                agent.save_model(model_name="best_model.pth")
        # end benchmarking
        print("Best Model Score: %.2f, std=%.2f" % (best_scores_info["mean"], best_scores_info["std"]))
    else:
        if not args.test:  # train the model without testing
            n_train_steps = args.running_steps // n_envs  # calculate the total steps of training
            agent.train(n_train_steps)  # train the model directly.
            agent.save_model("final_train_model.pth")  # save the final model file.
            print("Finish training!")
        else:  # test a trained model
            def env_fn():
                args_test = deepcopy(args)
                args_test.parallels = 1
                return make_envs(args_test)

            agent.render = True
            agent.load_model(agent.model_dir_load, args.seed)  # load the model file
            scores = agent.test(env_fn, args.test_episode)  # test the model
            print(f"Mean Score: {np.mean(scores)}, Std: {np.std(scores)}")
            print("Finish testing.")

    # the end.
    envs.close()  # close the environment
    agent.finish()  # finish the example

In [ ]:
# run(args)
envs = make_envs(args)  # create simulation environments
envs.action_space.n

In [ ]:
import xuance
runner = xuance.get_runner(method='dqn',
                           env='Box2D',
                           env_id='LunarLander-v2',
                           is_test=True,
                           config_path="./configs/basic.yaml")
print(type(runner))
runner.run()

In [ ]:
import xuance
runner = xuance.get_runner(method='MPE',
                           env='mpe',
                           env_id='simple_spread_v3',
                           is_test=True,
                           config_path="./configs/lular.yaml")
print(type(runner))
runner.run()